In [1]:
! pip install BeautifulSoup4

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests 
from bs4 import BeautifulSoup 

print('Libraries imported.')

Libraries imported.


In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
!pip install lxml  
soup = BeautifulSoup(r.content, 'html') 
table = soup.find('div', attrs = {'id':'container'}) 

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [4]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [5]:
# define the dataframe columns

column_names = ['PostalCode', 'Borough', 'Neighborhood'] # The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood


In [18]:
for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name}, ignore_index=True)

print(neighbors)

# skip not assigned boroughs:
df = neighbors.drop(neighbors[(neighbors.Borough == "Not assigned")].index)
# give "Not assigned" Neighborhoods same name as Borough:
neighbors.Neighborhood.replace("Not assigned", neighbors.Borough, inplace=True)

# copy Borough value to Neighborhood if NaN:
neighbors.Neighborhood.fillna(df.Borough, inplace=True)
# drop duplicate rows:
neighbors=neighbors.drop_duplicates()

neighbors.head()

    PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M6A        North York   
5          M7A  Downtown Toronto   
6          M9A         Etobicoke   
7          M1B       Scarborough   
8          M1B       Scarborough   
9          M3B        North York   
10         M4B         East York   
11         M4B         East York   
12         M5B  Downtown Toronto   
13         M5B  Downtown Toronto   
14         M6B        North York   
15         M9B         Etobicoke   
16         M9B         Etobicoke   
17         M9B         Etobicoke   
18         M9B         Etobicoke   
19         M9B         Etobicoke   
20         M1C       Scarborough   
21         M1C       Scarborough   
22         M1C       Scarborough   
23         M3C        North York   
24         M3C        North York   
25         M4C         East York   
26         M5C  Downtown Tor

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [16]:
neighbors.shape

(210, 3)